In [44]:
from qick import *
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [45]:
# Pyro to communicate with board
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [46]:
# Import dictionary of sequences from pickle file
import pickle
with open('Sequence_ch1_ch2_mw.pickle', 'rb') as handle:
    raman_cooling_sequences = pickle.load(handle)

In [47]:
# Set useful constants
GEN_CH_A = 1
GEN_CH_B = 0
RO_CH_C = 1
RO_CH_D = 0

In [48]:
# TODO: Extract parsing function and inherit it instead.
# TODO: Move into separate file.
class DigitalOutput():
    sequence = []

    def __init__(self, pmod_ch, seq):
        self.pmod_ch = pmod_ch
        self.seq = seq

        # Remove pre-exisiting sequence if it already exists for channel
        for l in self.sequence[:]:
            if l[-1] == self.pmod_ch:
                self.sequence.remove(l)
        
        if self.seq != None:
            # Convert list of tuples into list of lists
            self.seq = [list(t) for t in self.seq]

            # Convert pulse widths into elapsed time and append channel to list
            time = 0
            for l in self.seq:
                l[0], time = time, time + l[0]
                l.append(pmod_ch)

            delete_indices = []
            for i in range(len(self.seq) - 1):
                if self.seq[i][1] == self.seq[i+1][1]:
                    delete_indices.append(i+1)
            for i in reversed(delete_indices):
                del self.seq[i]
            
            # End final pulse
            self.seq.append([time, 0, pmod_ch])

            # Add channel sequence to master sequence and sort in order of time
            [self.sequence.append(l) for l in self.seq]
            self.sequence.sort(key=lambda x: x[0])
        
    def configure(self):
        out = 0
        for l in DigitalOutput.sequence: 
            time = int(self.us2cycles((l[0]) / 1e3))
            state = l[1]
            bit_position = l[2]

            if state == 1:
                out |= (1 << bit_position)
            elif state == 0:
                out &= ~(1 << bit_position)

            rp = 0 # tproc register page
            r_out = 31 # tproc register
            # print(bin(out), time)
            self.regwi(rp, r_out, out)
            self.seti(soccfg['tprocs'][0]['output_pins'][0][1], rp, r_out, time)

In [49]:
# TODO: Change into a class and inherit parsing.
# TODO: Move into separate file.
def analogue_sequence(seq):
    on_durations = []
    on_times = []
    on_freqs = []

    if seq != None:
        for l in seq:
            if int(l[1]) == 1:
                on_durations.append(int(l[0])/1e3)

        running_time = 0
        for l in seq:
            if int(l[1]) == 1:
                on_times.append(running_time/1e3)
                on_freqs.append(l[2]*1e3)
            running_time += l[0]
        finish_time = running_time/1e3
        
        if len(on_durations) != len(on_times):
            raise ValueError

    return on_durations, on_times, on_freqs, finish_time

on_durations, on_times, on_freqs, finish_time = analogue_sequence(raman_cooling_sequences["MW"])
sum = 0
for t in raman_cooling_sequences["MW"]:
    print(t[0])
    sum += t[0]
    print(sum)
    print("\n")

100.0
100.0


47000
47100.0


100.0
47200.0


3000.0
50200.0


450.0
50650.0


100.0
50750.0


450.0
51200.0


3000.0
54200.0


450.0
54650.0


100.0
54750.0


450.0
55200.0


3000.0
58200.0


450.0
58650.0


100.0
58750.0


450.0
59200.0


3000.0
62200.0


450.0
62650.0


100.0
62750.0


450.0
63200.0


3000.0
66200.0


450.0
66650.0


100.0
66750.0


450.0
67200.0


3000.0
70200.0


450.0
70650.0


100.0
70750.0


450.0
71200.0


3000.0
74200.0


450.0
74650.0


100.0
74750.0


450.0
75200.0


3000.0
78200.0


450.0
78650.0


100.0
78750.0


450.0
79200.0


3000.0
82200.0


450.0
82650.0


100.0
82750.0


450.0
83200.0


3000.0
86200.0


450.0
86650.0


100.0
86750.0


450.0
87200.0


3000.0
90200.0


450.0
90650.0


100.0
90750.0


450.0
91200.0


3000.0
94200.0


100.0
94300.0




In [50]:
prog = QickProgram(soccfg)
relax_delay = 0
t_offset = 38 # Offset DAC by 38 clock cycles for alignment with digital pulses

# Initialise DAC
prog.declare_gen(ch=GEN_CH_A, nqz=1)

# Set default pulse parameters
# TODO: Can set here and override later?
phase = prog.deg2reg(0, gen_ch=GEN_CH_A)
gain = 10000
prog.default_pulse_registers(ch=GEN_CH_A, phase=phase, gain=gain)

prog.synci(200)  # give processor some time to configure pulses

prog.regwi(0, 14, 9) # 10 reps, stored in page 0, register 14
prog.label("LOOP_I") # Start of internal loop

# Configure DAC pulses
for i in range(len(on_durations)):
    freq = prog.freq2reg(on_freqs[i], gen_ch=GEN_CH_A) # Calculate freq. of each pulse
    # TODO: Set phase and gain parameters for each pulse
    # phase = self.deg2reg(self.cfg["res_phase"], gen_ch=GEN_CH_A)
    # gain = self.cfg["pulse_gain"]
    prog.set_pulse_registers(ch=GEN_CH_A, freq=freq, style="const", length=prog.us2cycles(on_durations[i], gen_ch=GEN_CH_A)) # Store pulse freq. and length in register
    prog.pulse(ch=GEN_CH_A, t=prog.us2cycles(on_times[i])-t_offset) # Trigger pulse at given time
    print(on_times[i])

# Configure digital pulses
# TODO: First digital pulse in a sequence is ~22ns delayed
seq_0 = DigitalOutput(0, raman_cooling_sequences["ch2_Dig"])
seq_1 = DigitalOutput(1, raman_cooling_sequences["ch1_Dig"])
DigitalOutput.configure(prog)

# prog.synci(38586)

# Synchronise channels and go back to beginning of loop
prog.wait_all()
# prog.sync_all(prog.us2cycles(relax_delay))
prog.synci(prog.us2cycles(94.3))
prog.loopnz(0, 14, "LOOP_I")
prog.end()

# Run program!
prog.config_all(soc)
soc.tproc.start()
print(prog) # Print assembly instructions

0.1
50.65
54.65
58.65
62.65
66.65
70.65
74.65
78.65
82.65
86.65
90.65

// Program

        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 10000;                    //gain = 10000
        synci 200;
        regwi 0, $14, 9;
LOOP_I: regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 618701;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 28877 
        regwi 1, $27, 3;                        //t = 3
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 589885;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
        regwi 1, $27, 20708;                    //t = 20708
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 24029867;                 //freq = 24029867
        regwi 1, $26, 589885;                   //phrst| stdysel | mode